In [1]:
prog drop _all

*! Scott Cunningham use - scuse v1.0.1  BB29
prog scuse
	version 14.2
	syntax anything [, CLEAR noDesc]
	if regexm("`anything'", ".+.zip$") {
		capt copy "https://storage.googleapis.com/causal-inference-mixtape.appspot.com/`anything'" ., replace
		if _rc != 0 {
			di as err _n "Error: cannot copy `anything' to current working directory."
			exit 
		}
		qui unzipfile "`anything'", replace
		loc fn = reverse(substr(reverse("`anything'"), 5, .))
		capt use `fn', `clear'
	}
	else {
		capt use "https://storage.googleapis.com/causal-inference-mixtape.appspot.com/`anything'", `clear'
	}
	if _rc != 0 {
 		if _rc == 4 {
 			di as err _n "Error: data in memory would be lost. Use  scuse `anything', clear  to discard changes."
 		} 
 		else if _rc == 610 {
 			di as err _n "Error: C datafile `anything' must be accessed from Stata 14.x." _n  "It cannot be read by earlier Stata versions."
 		}
 		else {
		di as err _n "Error: C datafile `anything' does not exist. Contents of memory not altered."
		}
		exit
	}
	if "`desc'" != "nodesc" { 
		describe
	}
end
// 1.0.1: Cloned -bcuse.ado- and made it -scuse.ado-

In [2]:
scuse card


Contains data from https://storage.googleapis.com/causal-inference-mixtape.appspot.com/card.dta
  obs:         3,010                          
 vars:            37                          18 May 2018 18:18
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
              storage   display    value
variable name   type    format     label      variable label
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
id              float   %9.0g                 
nearc2          float   %9.0g                 
nearc4          float   %9.0g                 
educ            float   %9.0g        

In [ ]:
global y lwage
global s 

In [3]:
reg lwage educ exper black south married smsa


      Source |       SS           df       MS      Number of obs   =     3,003
-------------+----------------------------------   F(6, 2996)      =    219.15
       Model |  180.255137         6  30.0425229   Prob > F        =    0.0000
    Residual |  410.705979     2,996  .137084773   R-squared       =    0.3050
-------------+----------------------------------   Adj R-squared   =    0.3036
       Total |  590.961117     3,002  .196855802   Root MSE        =    .37025

------------------------------------------------------------------------------
       lwage |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        educ |   .0711729   .0034824    20.44   0.000     .0643447     .078001
       exper |   .0341518   .0022144    15.42   0.000     .0298098    .0384938
       black |  -.1660274   .0176137    -9.43   0.000    -.2005636   -.1314913
       south |  -.1315518   .0149691    -8.79   0.

In [4]:
ivregress 2sls lwage (educ=nearc4) exper black south married smsa, first


First-stage regressions
-----------------------

                                                Number of obs     =      3,003
                                                F(   6,   2996)   =     456.14
                                                Prob > F          =     0.0000
                                                R-squared         =     0.4774
                                                Adj R-squared     =     0.4764
                                                Root MSE          =     1.9373

------------------------------------------------------------------------------
        educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       exper |   -.404434   .0089402   -45.24   0.000    -.4219636   -.3869044
       black |  -.9475281   .0905256   -10.47   0.000    -1.125027   -.7700295
       south |  -.2973528   .0790643    -3.76   0.000    -.4523787   -.1423269
 

In [5]:
reg educ nearc4 exper black south married smsa


      Source |       SS           df       MS      Number of obs   =     3,003
-------------+----------------------------------   F(6, 2996)      =    456.14
       Model |  10272.0963         6  1712.01605   Prob > F        =    0.0000
    Residual |  11244.7835     2,996  3.75326552   R-squared       =    0.4774
-------------+----------------------------------   Adj R-squared   =    0.4764
       Total |  21516.8798     3,002  7.16751492   Root MSE        =    1.9373

------------------------------------------------------------------------------
        educ |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
      nearc4 |   .3272826   .0824239     3.97   0.000     .1656695    .4888957
       exper |   -.404434   .0089402   -45.24   0.000    -.4219636   -.3869044
       black |  -.9475281   .0905256   -10.47   0.000    -1.125027   -.7700295
       south |  -.2973528   .0790643    -3.76   0.

In [6]:
 test nearc4


 ( 1)  nearc4 = 0

       F(  1,  2996) =   15.77
            Prob > F =    0.0001
